In [75]:
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

plt.rcParams['font.family'] = 'Malgun Gothic'

In [60]:
# 데이터를 train과 test로 나눔
# 가장 많이 사용된 단어 1000개 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

In [61]:
# 패딩
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [62]:
# train과 validation 분리
val_n = round(len(x_train)*0.3)

x_val = x_train[:val_n] # train의 30%를 validation으로 사용
x_train = x_train[val_n:]
y_val = y_train[:val_n] # train의 30%를 validation으로 사용
y_train = y_train[val_n:]

In [68]:
# 모델 적용
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression()

solver : 최적화에 사용할 알고리즘 결정('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')<br>
class_weight : 클래스에 대한 가중치들의 값<br>
max_iter : solver가 수렴하게 만드는 최대 반복횟수<br>
C : cost function, L1 혹은 L2 제약조건의 강도 설정<br>
L1 규제 : 손실함수에 가중치의 절대값을 더하는 방법(이동거리 = 벡터 성분 각각을 더한 값)  
L2 규제 : 손실함수에 가중치에 대한 L2 norm을 더함(이동거리를 유클리디안 거리로 계산)

In [69]:
lr_preds = classifier.predict(x_val)

print('accuracy: {:0.3f}'.format(accuracy_score(y_val, lr_preds)))
print('Precision: {:0.3f}'.format(precision_score(y_val, lr_preds, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_val, lr_preds, average='macro')))
#print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , lr_preds, multi_class='ovo')))

accuracy: 0.314
Precision: 0.063
recall: 0.063


In [70]:
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import GridSearchCV

params={'C':[0.01, 0.1, 1, 5, 10]}

grid_clf = GridSearchCV(classifier, param_grid=params, scoring='accuracy', cv=3)
grid_clf.fit(x_train, y_train)

print('최적 하이퍼 파라미터:{0}, 최적 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

최적 하이퍼 파라미터:{'C': 0.1}, 최적 정확도:0.303


GridSearchCV : 하이퍼파라미터를 순차적으로 입력해 학습을 하고 측정을 하면서 가장 좋은 파라미터를 알려줌<br>
param_grid : 파라미터 딕셔너리(파라미터 명과 사용될 여러 파라미터 값 지정)  
cv : 교차 검증을 위해 분할되는 폴드 수

In [71]:
# gridsearch로 찾은 최적 하이퍼파라미터 사용
lr_clf_new = LogisticRegression(C=0.1)
lr_clf_new.fit(x_train, y_train)
pred_new = lr_clf_new.predict(x_val)
print('accuracy: {:.3f}'.format(accuracy_score(y_val, pred_new)))
print('Precision: {:0.3f}'.format(precision_score(y_val, pred_new, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_val, pred_new, average='macro')))

accuracy: 0.322
Precision: 0.063
recall: 0.064


In [79]:
# test결과
lr_preds_t = classifier.predict(x_test)

print('accuracy: {:0.3f}'.format(accuracy_score(y_test, lr_preds_t)))
print('Precision: {:0.3f}'.format(precision_score(y_test, lr_preds_t, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_test, lr_preds_t, average='macro')))

accuracy: 0.321
Precision: 0.079
recall: 0.075


In [78]:
# 모델 사용결과
xhat_idx = np.random.choice(x_test.shape[0], 15)
xhat = x_test[xhat_idx]
yhat = lr_clf_new.predict(xhat)

num=0
for i in range(15):
    print('True : ' + str(y_test[xhat_idx[i]]) + ', Predict : ' + str(yhat[i]))
    if str(y_test[xhat_idx[i]]) == str(yhat[i]):
        num+=1

print("15개 중 모델이 일치한 것의 개수 : ", num)

True : 3, Predict : 3
True : 23, Predict : 3
True : 3, Predict : 3
True : 4, Predict : 3
True : 19, Predict : 15
True : 20, Predict : 3
True : 3, Predict : 3
True : 35, Predict : 3
True : 3, Predict : 4
True : 3, Predict : 3
True : 3, Predict : 3
True : 19, Predict : 40
True : 28, Predict : 38
True : 3, Predict : 3
True : 9, Predict : 4
15개 중 모델이 일치한 것의 개수 :  6
